#### Estimate the amount of missing sediment input in Canada Basin using the turbocline depth

In [1]:
import numpy as np
import netCDF4 as nc
import glob

##### Parameters

In [2]:
imin, imax = 1479, 2179
jmin, jmax = 159, 799
isize, jsize = imax-imin, jmax-jmin

###### Load files

In [3]:
mask  = nc.Dataset('/ocean/brogalla/GEOTRACES/data/ANHA12/ANHA12_mesh1.nc')
tmask = np.array(mask.variables['tmask'])

In [5]:
# ANHA12 dynamics fields accessible upon request via http://knossos.eas.ualberta.ca/anha/anhatable.php
file_list = np.sort(glob.glob('/data/brogalla/ANHA12/ANHA12-EXH006_5d_gridT_y2015*'))

##### Calculations

In [6]:
# Find indices to separate out Canada Basin:

x_ind = np.arange(1621, 2100, 1)
y_ind = (-7/8)*x_ind + 1517 + 700

CB_indx = []
CB_indy = []

for index in range(0,len(x_ind)):
    CB_x = np.arange(x_ind[index],2179,1)
    CB_y = np.ones(CB_x.shape)*y_ind[index]
    
    CB_indx = np.append(CB_x, CB_indx)
    CB_indy = np.append(CB_y, CB_indy)

In [7]:
# Separate Canada Basin:
mask_CB  = np.zeros((isize,jsize))
for i, j in zip(CB_indx, CB_indy):
    mask_CB[int(i-imin),int(j-jmin)]  = 1
    
mask_CB[150:-1 ,-8:-1] = 1

In [8]:
turbo_maxs = np.zeros((len(file_list),1))
turbo_mins = np.zeros((len(file_list),1))
turbo_aves = np.zeros((len(file_list),1))

for i, file in enumerate(file_list):
    ds = nc.Dataset(file)
    
    # Load turbocline depth variable and mask land values
    turbo   = np.array(ds.variables['sohmld'])[0,imin:imax,jmin:jmax]
    turbo_m = np.ma.masked_where((tmask[0,0,imin:imax,jmin:jmax] < 0.1) & (mask_CB==0), turbo)
    
    turbo_maxs[i] = np.ma.amax(turbo_m)
    turbo_mins[i] = np.ma.amin(turbo_m[turbo_m > 0]) # don't want to bias by zero
    turbo_aves[i] = np.ma.mean(turbo_m)

In [10]:
print(f'Maximum turbocline depth seen in 2015: {np.amax(turbo_maxs):.2f}')
print(f'Minimum turbocline depth seen in 2015: {np.amin(turbo_mins):.2f}')
print(f'Average turbocline depth seen in 2015: {np.average(turbo_aves):.2f}')

Maximum turbocline depth seen in 2015: 170.29
Minimum turbocline depth seen in 2015: 10.00
Average turbocline depth seen in 2015: 21.45


Now, to calculate the missing contribution in grams of sediment per meter squared:

In [12]:
turbo_max = np.amax(turbo_maxs)
turbo_min = np.amin(turbo_mins)
turbo_ave = np.average(turbo_aves)

In [13]:
# missing Mn (nM) * molar mass Mn * L --> m3 * kg of sed --> g of sed * turbocline depth  / (Mn frac in sediments * solubility)
max_missing = (4e-9*54.938*1e3*1e3*turbo_max)/(0.270*0.65)
min_missing = (4e-9*54.938*1e3*1e3*turbo_min)/(0.270*0.65)
ave_missing = (4e-9*54.938*1e3*1e3*turbo_ave)/(0.270*0.65)

In [14]:
print(f'Maximum missing in 2015: {max_missing:.2f} g/m2')
print(f'Minimum missing in 2015: {min_missing:.2f} g/m2')
print(f'Average missing in 2015: {ave_missing:.2f} g/m2')

Maximum missing in 2015: 213.23 g/m2
Minimum missing in 2015: 12.52 g/m2
Average missing in 2015: 26.86 g/m2
